## SFT train the d20 model

I'm going to approach this like `challenge-28-midtrain-d20/midtrain-d20.ipynb`. I'll use the 8xH100 in lambda cloud and run all the scripts from this notebook.

Follow the instructions here to get the machine ready: `challenge-28-midtrain-d20/midtrain-d20.ipynb`

In [ ]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

#### Repeat CORE eval on base model

For reasons discussed in `challenge-28-midtrain-d20/investigate-questions.ipynb`, I want to first do the CORE eval on the base model **with max-per-task=500**

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_base_eval -- \
--source=base --model-tag=d20 --max-per-task=500

#### Redo mid training

Because I left out updating muon momentum (noticed in `challenge-29-understand-sft/understand-sft.ipynb`), I want to redo the mid training.

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_mid_train -- \
--model_tag=d20 --run=challenge-30-1

#### Do chat eval on mid model

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=mid --model-tag=d20

#### SFT train

First do a few iterations to make sure everything works.

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model-tag=d20 --num_iterations=10 --run=challenge-30-2

Then do for real:

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model-tag=d20 --run=challenge-30-3

#### Do chat eval on sft model

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20

(while chat eval is running, remember to sftp the mid and sft checkpoints to my mac)